In [1]:
! pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import Keys
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import random
import json
import pandas as pd

In [2]:
search_name = "Духи"
articul_num = "1713520"
post_place = "г Москва, Улица Лазо 4к1"

In [74]:
_bot = bot
bot = Bot(driver=_bot.driver)

In [2]:
# Python Modules
import os
import json
import logging
import pandas as pd

# For telegram api
# pip install python-telegram-bot --upgrade
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext, \
    CallbackQueryHandler, TypeHandler, ConversationHandler



In [3]:
%run Bot.py

BOTS_NAME = ['Oleg', 'Einstein', 'Boulevard Depo']

file_path = "users_data/794329884_order.xlsx"
inside=True
df = pd.read_excel(file_path)

orders = [row.tolist() for i, row in df.iterrows()]

articles = [order[0] for order in orders]
watch_bot = Bot(name="Watcher")
for i, article in enumerate(articles):
    orders[i] += [watch_bot.get_data_cart(article)]

max_bots = max([order[3] for order in orders])
bots = [Bot(name=BOTS_NAME[i]) for i in range(max_bots)]
data_for_bots = [[] for _ in range(max_bots)]

for _, order in enumerate(orders):
    if inside:
        article, search_key, quantity, pvz_cnt, _, __, additional_data = order
    else:
        article, search_key, quantity, pvz_cnt, additional_data = order

    for i in range(max_bots):
        if pvz_cnt > 0:
            data_for_bots[i] += [[article, search_key, quantity, additional_data]]
        pvz_cnt -= 1

post_places = [order[5] for order in orders][:max_bots]
order_id = str(orders[0][4])

report = []
for i, bot in enumerate(bots):
    print('run bot buy')
    report += bot.buy(data_for_bots[i], post_places[i], order_id)

run bot buy
price 1522
price_filter started
min_price 1100 max_price 1900
price_filter ended
['44398622']
['44398622']


WebDriverException: Message: chrome not reachable
  (Session info: chrome=100.0.4896.88)
Stacktrace:
Backtrace:
	Ordinal0 [0x00679943+2595139]
	Ordinal0 [0x0060C9F1+2148849]
	Ordinal0 [0x005043F0+1065968]
	Ordinal0 [0x004F87C2+1017794]
	Ordinal0 [0x004F8FF8+1019896]
	Ordinal0 [0x004FA892+1026194]
	Ordinal0 [0x004F4219+999961]
	Ordinal0 [0x00505860+1071200]
	Ordinal0 [0x0055B2D2+1422034]
	Ordinal0 [0x0054B806+1357830]
	Ordinal0 [0x00526086+1204358]
	Ordinal0 [0x00526F96+1208214]
	GetHandleVerifier [0x0081B232+1658114]
	GetHandleVerifier [0x008D312C+2411516]
	GetHandleVerifier [0x0070F261+560433]
	GetHandleVerifier [0x0070E366+556598]
	Ordinal0 [0x0061286B+2173035]
	Ordinal0 [0x006175F8+2192888]
	Ordinal0 [0x006176E5+2193125]
	Ordinal0 [0x006211FC+2232828]
	BaseThreadInitThunk [0x7683FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77207A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77207A4E+238]


In [34]:
min_price, max_price = 1141, 1902

print("price_filter started")
print("min_price", min_price, "max_price", max_price)
min_price_field = bot.catalog.min_price_field()
min_price_field.click()
min_price_field.send_keys(Keys.CONTROL + "a")
min_price_field.send_keys(Keys.DELETE)
min_price_field.send_keys(str(min_price))

sleep(1)

max_price_field = bot.catalog.max_price_field()
max_price_field.click()

sleep(1)
max_price_field = bot.catalog.max_price_field()
max_price_field.click()
max_price_field.click()
max_price_field.send_keys(Keys.CONTROL + "a")
max_price_field.send_keys(Keys.DELETE)
max_price_field.send_keys(str(max_price))
max_price_field.send_keys(Keys.ENTER)
print("price_filter ended")

price_filter started
1141 1902
min_price_field =  <selenium.webdriver.remote.webelement.WebElement (session="4cf82b9719ef4987aa066532788b8c3d", element="6b57a28b-6676-4fa1-a6fc-9c15762332af")>
price_filter ended


In [10]:
def choose_post_place(adress):
    bot.driver.find_element(By.XPATH, '//h2[text()="Способ доставки"]/../../div[text()="Выбрать адрес доставки"]').click()
    sleep(1)
    bot.driver.find_element(By.XPATH, '//button[text()="Выбрать адрес доставки"]').click()
    sleep(2)
    bot.driver.find_element(By.XPATH, '//input[@placeholder="Введите адрес"]').send_keys(adress)
    sleep(2)
    bot.driver.find_element(By.XPATH, '//ymaps[text()="Найти"]').click()
    sleep(2)
    try:
        bot.driver.find_element(By.XPATH, '//ymaps[contains(@class, "__first")]').click()
    except:
        pass
    sleep(2)
    bot.driver.find_element(By.XPATH, '//span[contains(text(), "'+ adress +'")]').click()
    sleep(2)
    bot.driver.find_element(By.XPATH, '//div[@class="balloon-content-block"]/button').click()
    sleep(2)
    bot.driver.find_element(By.XPATH, '//button[@class="popup__btn-main"]').click()
    sleep(2)

In [17]:
choose_post_place(post_places[0])

In [13]:
import random

In [20]:
self = bot
number = '9104499982'
def get_qr_code(file_name):
    try:
        self.driver.find_element(By.XPATH, '//button[text()="                Оплатить заказ                "]').click()
        sleep(2)
        self.driver.find_element(By.XPATH, '//button[contains(@class,"popup__btn-main")]').click()
        sleep(2)
        svg = self.driver.find_element(By.XPATH, '//div[@class="qr-code__value"]')
        self.save_qr_code(svg, file_name)
        sleep(2)
        self.driver.find_element(By.CLASS_NAME, 'popup__close').click()
    except:
#         Utils.login(number)
        login(self.driver, number)
        input("Нажми Enter, зайка")
        get_qr_code(file_name)

In [21]:
get_qr_code('file_name.svg')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".j-main-login"}
  (Session info: chrome=100.0.4896.75)
Stacktrace:
Backtrace:
	Ordinal0 [0x01039943+2595139]
	Ordinal0 [0x00FCC9F1+2148849]
	Ordinal0 [0x00EC4528+1066280]
	Ordinal0 [0x00EF0FD4+1249236]
	Ordinal0 [0x00EF11CB+1249739]
	Ordinal0 [0x00F1D812+1431570]
	Ordinal0 [0x00F0BA34+1358388]
	Ordinal0 [0x00F1BAF2+1424114]
	Ordinal0 [0x00F0B806+1357830]
	Ordinal0 [0x00EE6086+1204358]
	Ordinal0 [0x00EE6F96+1208214]
	GetHandleVerifier [0x011DB232+1658114]
	GetHandleVerifier [0x0129312C+2411516]
	GetHandleVerifier [0x010CF261+560433]
	GetHandleVerifier [0x010CE366+556598]
	Ordinal0 [0x00FD286B+2173035]
	Ordinal0 [0x00FD75F8+2192888]
	Ordinal0 [0x00FD76E5+2193125]
	Ordinal0 [0x00FE11FC+2232828]
	BaseThreadInitThunk [0x763BFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77787A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77787A4E+238]


In [8]:
def login(driver, number):
        # vars # 1
        to_login_btn = driver.find_element(By.CLASS_NAME, 'j-main-login')

        # resolve # 1
        to_login_btn.click()
        sleep(random.uniform(3, 5))
        
        # vars # 2
        login_input = driver.find_element(By.CLASS_NAME, 'input-item')
        login_btn = driver.find_element(By.CLASS_NAME, 'login__btn')

        # resolve # 2
        login_input.send_keys(number)
        sleep(random.uniform(2, 4))
        login_btn.click()

In [ ]:
[]+[{'a': '2'}]

In [ ]:
def transform_price(price):
    price = str(price)
    if len(price)>=4:
        price = price[:-2] + "00"
    elif len(price)>1:
        price = price[:-1] + "0"
    return price

In [ ]:
transform_price(12222)

In [ ]:
! pip install ujson
! pip install uvloop
! pip install cchardet
! pip install aiodns
! pip install aiohttp[speedups]
! pip install aiogram

In [ ]:
import logging
from aiogram import Bot, Dispatcher, executor, types


[{'a': '2'}]

In [44]:
def transform_price(price):
    price = str(price)
    if len(price)>=4:
        price = price[:-2] + "00"
    elif len(price)>1:
        price = price[:-1] + "0"
    return price

In [46]:
transform_price(12222)

'12200'

In [5]:
! pip install ujson
! pip install uvloop
! pip install cchardet
! pip install aiodns
! pip install aiohttp[speedups]
! pip install aiogram

    ERROR: Command errored out with exit status 1:
     command: 'C:\ProgramData\Anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Astrocatling\\AppData\\Local\\Temp\\pip-install-0iy9qm68\\uvloop_c88c366010154bb38712f888667ddaa9\\setup.py'"'"'; __file__='"'"'C:\\Users\\Astrocatling\\AppData\\Local\\Temp\\pip-install-0iy9qm68\\uvloop_c88c366010154bb38712f888667ddaa9\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\Astrocatling\AppData\Local\Temp\pip-pip-egg-info-660ima5t'
         cwd: C:\Users\Astrocatling\AppData\Local\Temp\pip-install-0iy9qm68\uvloop_c88c366010154bb38712f888667ddaa9\
    Complete output (5 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in 

    ----------------------------------------
    ERROR: Command errored out with exit status 1:
     command: 'C:\ProgramData\Anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Astrocatling\\AppData\\Local\\Temp\\pip-install-0iy9qm68\\uvloop_d5f12cf8d2604ebba92c8522b6f80197\\setup.py'"'"'; __file__='"'"'C:\\Users\\Astrocatling\\AppData\\Local\\Temp\\pip-install-0iy9qm68\\uvloop_d5f12cf8d2604ebba92c8522b6f80197\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\Astrocatling\AppData\Local\Temp\pip-pip-egg-info-dy95s157'
         cwd: C:\Users\Astrocatling\AppData\Local\Temp\pip-install-0iy9qm68\uvloop_d5f12cf8d2604ebba92c8522b6f80197\
    Complete output (5 lines):
    Traceback (most recent c


  Running setup.py clean for uvloop
Failed to build uvloop
    Running setup.py install for uvloop: started
    Running setup.py install for uvloop: finished with status 'error'


In [7]:
import logging
from aiogram import Bot, Dispatcher, executor, types
